In [ ]:
from tensorflow.keras import models, layers
from tensorflow import keras
from data_handler import get_selected_data
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np

In [ ]:
data = get_selected_data()

In [ ]:
trajectories = [tmd["traj_data"] for tmd in data]
classes = [tmd["class"] for tmd in data]

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(classes)
for cls, lbl in zip(le.classes_, le.transform(le.classes_)):
    print(lbl, cls)
labeled_classes = le.transform(classes)
print("\nAll classes labeled:", labeled_classes)

In [ ]:
def preprocess_data(trajectories, classes):
    max_length = max(t.shape[0] for t in trajectories)
    X, y = [], []
    for traj, cls in zip(trajectories, classes):
        left = max_length - traj.shape[0]
        tt = np.zeros((max_length, 3))
        tt[left:,:] = traj
        yy = np.zeros((max_length,))
        yy[left:] = cls
        X.append(tt)
        y.append(yy)
    return X, y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    trajectories, labeled_classes, stratify=labeled_classes, random_state=0, test_size=0.25,
)

X_validation, X_test = X_test[:200], X_test[200:]
y_validation, y_test = y_test[:200], y_test[200:]

In [ ]:
def prepare_data(trajs, cls, hist_size=75):
    X, y = [], []
    for traj, cls in zip(trajs, cls):
        traj_len = traj.shape[0]
        for i in range(traj_len - hist_size):
            X.append(traj[i:i+hist_size])
            _y = [0] * le.classes_.shape[0]
            _y[cls] = 1
            y.append(_y)
    return np.array(X), np.array(y)

In [ ]:
last_n_points = 75
tX, ty = prepare_data(X_train[:50], y_train[:50], last_n_points)

In [ ]:
model = keras.Sequential()
model.add(layers.LSTM(1, input_shape=(last_n_points, 3)))
model.add(layers.Dense(le.classes_.shape[0]))
opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='mse', metrics=['accuracy'], optimizer=opt)
model.summary()
model.fit(tX, ty, epochs=2)